In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Conv2D, Dropout, Dense, Flatten, BatchNormalization, MaxPooling2D, LeakyReLU
from keras.optimizers import RMSprop, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.utils.np_utils import to_categorical

from sklearn.model_selection import train_test_split

In [3]:
test = pd.read_csv("./input/Kannada-MNIST/test.csv")
train = pd.read_csv("./input/Kannada-MNIST/train.csv")

x = train.iloc[:,1:].values
y = train.iloc[:,0].values

x = x.reshape(x.shape[0], 28, 28, 1)
y = to_categorical(y, 10)

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size = 0.10, random_state=6666) 

train_datagen = ImageDataGenerator(rescale=1./255.,
                                   rotation_range=10,
                                   width_shift_range=0.25,
                                   height_shift_range=0.25,
                                   shear_range=0.1,
                                   zoom_range=0.25,
                                   horizontal_flip=False)

valid_datagen = ImageDataGenerator(rescale=1./255.)

In [4]:
batch_size = 1024
num_classes = 10
epochs = 100
learning_rate = 0.001

def new_model():

    model = Sequential()
    model.add(Flatten(input_shape=(28, 28, 1)))
    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))

    model.add(Dense(256))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    
    model.add(Dense(64))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))

    model.add(Dense(10, activation='softmax'))
    return model

model = new_model()
model.summary(120)

optimizer = Adam(lr=learning_rate)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

Model: "sequential_1"
________________________________________________________________________________________________________________________
Layer (type)                                          Output Shape                                    Param #           
flatten_1 (Flatten)                                   (None, 784)                                     0                 
________________________________________________________________________________________________________________________
dense_1 (Dense)                                       (None, 512)                                     401920            
________________________________________________________________________________________________________________________
batch_normalization_1 (BatchNormalization)            (None, 512)                                     2048              
________________________________________________________________________________________________________________________
leaky_re_l

In [5]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=200,
                                            verbose=1,
                                            factor=0.2)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)

history = model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
                              epochs=epochs,
                              validation_data=valid_datagen.flow(x_valid, y_valid),
                              callbacks=[learning_rate_reduction, es])


Epoch 1/100
53/53 [==============================] - 9s 161ms/step - loss: 1.2279 - accuracy: 0.6210 - val_loss: 1.4093 - val_accuracy: 0.8445
Epoch 2/100
53/53 [==============================] - 8s 158ms/step - loss: 0.5006 - accuracy: 0.8556 - val_loss: 0.7402 - val_accuracy: 0.8430
Epoch 3/100
53/53 [==============================] - 8s 158ms/step - loss: 0.3600 - accuracy: 0.8886 - val_loss: 0.1820 - val_accuracy: 0.9123
Epoch 4/100
53/53 [==============================] - 8s 155ms/step - loss: 0.2972 - accuracy: 0.9048 - val_loss: 0.2530 - val_accuracy: 0.9347
Epoch 5/100
53/53 [==============================] - 8s 156ms/step - loss: 0.2660 - accuracy: 0.9144 - val_loss: 0.0784 - val_accuracy: 0.9568
Epoch 6/100
53/53 [==============================] - 8s 156ms/step - loss: 0.2430 - accuracy: 0.9206 - val_loss: 0.0842 - val_accuracy: 0.9623
Epoch 7/100
53/53 [==============================] - 8s 155ms/step - loss: 0.2304 - accuracy: 0.9242 - val_loss: 0.0286 - val_accuracy: 0.959